In [ ]:
import pandas as pd

df = pd.read_csv("./script/FONSEScripMaster.txt", dtype=str)

data = df[["Token", "InstrumentName", "ShortName", "Series", "ExpiryDate", "StrikePrice", "OptionType", "BasePrice", "ExchangeCode"]]

data = data[data["Series"] == "OPTION"]
data = data[data["InstrumentName"] == "OPTSTK"]

# Print first few rows
print(data.head())

# only when it's needed to get latest data from the broker
# unique_instruments = data["ExchangeCode"].unique()
# for instrument in unique_instruments:
#     instrument_data = data[data["ExchangeCode"] == instrument]
#     file_name = f"./Optiondata/{instrument}.xlsx"
#     instrument_data.to_excel(file_name, index = False)


In [ ]:
import threading
from flask import Flask, request, jsonify, redirect, session
from breeze_connect import BreezeConnect
import urllib.parse

app = Flask(__name__)
app.secret_key = "1997"

API_KEY = "4_8463B9784119l72yC4426912px346P"
API_SECRET = "816O1S=eG215716g518CM)X225Z0!78b"
ACCESS_TOKEN = None

breeze = BreezeConnect(api_key=API_KEY)

@app.route('/')
def login():
    login_url = f"https://api.icicidirect.com/apiuser/login?api_key={urllib.parse.quote_plus(API_KEY)}"
    return redirect(login_url)

@app.route('/callback', methods=['POST'])
def callback():
    global ACCESS_TOKEN

    session_token = request.args.get("apisession")
    if not session_token:
        return "Authorization failed", 400

    try:
        breeze.generate_session(api_secret=API_SECRET, session_token=session_token)
        ACCESS_TOKEN = session_token
        session["access_token"] = ACCESS_TOKEN
        
        # Fetch customer details
        customer_details = breeze.get_customer_details(api_session=ACCESS_TOKEN)
        return jsonify({"message": "Session active", "customer_details": customer_details})
    except Exception as e:
        return jsonify({"error": str(e)}), 500

# Start Flask in a separate thread
def run_flask():
    app.run(debug=True, use_reloader=False)

threading.Thread(target=run_flask).start()

 * Serving Flask app '__main__'


 * Debug mode: on


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [08/Mar/2025 22:48:59] "GET / HTTP/1.1" 302 -
127.0.0.1 - - [08/Mar/2025 22:49:36] "POST /callback?apisession=50830829 HTTP/1.1" 200 -


###### 
* product_type:- "futures", "options", "furtureplus", "futureplus_sltp", "optionplus", "cash", "eatm", "btst", "margin", "marginplus"
* exchange_code:- "NSE", "NFO", "BSE"
* right:- 	"call", "put", "others"


In [19]:
from datetime import datetime

# Function to format date and datetime in ISO format
def format_iso(date_str, time_str="07:00:00"):
    return f"{datetime.strptime(date_str, '%d/%m/%Y').date()}T{time_str}.000Z"

start_date = format_iso("31/10/2024", "09:15:00")
end_date = format_iso("31/10/2024", "15:30:00")
exp_date = format_iso("31/10/2024", "15:30:00")

stock_code = "RELIND"
exchange_code = "NFO"
interval = "30minute"
strike_price="1300"
product_type = "options"
right = "call"

data = breeze.get_historical_data(
    interval = interval,
    from_date = start_date,
    to_date = end_date,
    expiry_date = exp_date,
    stock_code = stock_code,
    strike_price = strike_price,
    product_type = product_type,
    exchange_code = exchange_code,
    right=right
)

print(data)

{'Success': [{'datetime': '2024-10-31 09:15:00', 'stock_code': 'RELIND', 'exchange_code': 'NFO', 'product_type': 'Options', 'expiry_date': '31-OCT-24', 'right': 'Call', 'strike_price': '1300', 'open': '43', 'high': '43', 'low': '30.1', 'close': '30.75', 'volume': '14500', 'open_interest': '202000', 'count': 0}, {'datetime': '2024-10-31 09:45:00', 'stock_code': 'RELIND', 'exchange_code': 'NFO', 'product_type': 'Options', 'expiry_date': '31-OCT-24', 'right': 'Call', 'strike_price': '1300', 'open': '31.7', 'high': '36.35', 'low': '31.7', 'close': '34.5', 'volume': '10000', 'open_interest': '194000', 'count': 1}, {'datetime': '2024-10-31 10:15:00', 'stock_code': 'RELIND', 'exchange_code': 'NFO', 'product_type': 'Options', 'expiry_date': '31-OCT-24', 'right': 'Call', 'strike_price': '1300', 'open': '35.85', 'high': '37.8', 'low': '35.85', 'close': '37.45', 'volume': '1500', 'open_interest': '191000', 'count': 2}, {'datetime': '2024-10-31 10:45:00', 'stock_code': 'RELIND', 'exchange_code': '

In [20]:
import pandas as pd

df = pd.DataFrame(data['Success'])

# Display the DataFrame
print(df)

df.to_excel("Reliance.xlsx", index=False)

               datetime stock_code exchange_code product_type expiry_date  \
0   2024-10-31 09:15:00     RELIND           NFO      Options   31-OCT-24   
1   2024-10-31 09:45:00     RELIND           NFO      Options   31-OCT-24   
2   2024-10-31 10:15:00     RELIND           NFO      Options   31-OCT-24   
3   2024-10-31 10:45:00     RELIND           NFO      Options   31-OCT-24   
4   2024-10-31 11:15:00     RELIND           NFO      Options   31-OCT-24   
5   2024-10-31 11:45:00     RELIND           NFO      Options   31-OCT-24   
6   2024-10-31 12:15:00     RELIND           NFO      Options   31-OCT-24   
7   2024-10-31 12:45:00     RELIND           NFO      Options   31-OCT-24   
8   2024-10-31 13:15:00     RELIND           NFO      Options   31-OCT-24   
9   2024-10-31 13:45:00     RELIND           NFO      Options   31-OCT-24   
10  2024-10-31 14:15:00     RELIND           NFO      Options   31-OCT-24   
11  2024-10-31 14:45:00     RELIND           NFO      Options   31-OCT-24   